In [40]:
%run sources/slicer.py
%run sources/time_series_to_intervals.py
%run sources/time_series_event_merge.py

main_value = 1 #variable
main_feature = "patient"


In [41]:
feature = "distance"
p = pd.read_csv(f"processed_data/diabetes/diabetes_all_clean.csv")
set(p["label"])

{'Hypoglycemic symptoms',
 'Less-than-usual exercise activity',
 'Less-than-usual meal ingestion',
 'More-than-usual exercise activity',
 'More-than-usual meal ingestion',
 'NPH insulin dose',
 'Post-breakfast blood glucose measurement',
 'Post-lunch blood glucose measurement',
 'Post-supper blood glucose measurement',
 'Pre-breakfast blood glucose measurement',
 'Pre-lunch blood glucose measurement',
 'Pre-snack blood glucose measurement',
 'Pre-supper blood glucose measurement',
 'Regular insulin dose',
 'Typical exercise activity',
 'Typical meal ingestion',
 'UltraLente insulin dose',
 'Unspecified blood glucose measurement',
 'Unspecified special event'}

In [42]:
time_feature = "vt"

timeseries_df_list = {"paths": [],
                       "group_features": [],
                       "bucket_values": []}

event_df_list = {"paths": ["processed_data/diabetes/diabetes_all_clean.csv"],
                  "group_features": ["label"],
                  "value_feature": ["value"]}

slicer_params = {"stride": 12, "ow": 12, "ww": 4, "pw": 12, "granularity": "hours"} #variable

final_odf, final_pdf = samples_from_dataset(main_value, main_feature, time_feature, timeseries_df_list, event_df_list, slicer_params)

100%|██████████| 268/268 [00:00<00:00, 1678.07it/s]


In [43]:
final_pdf['label'][4]

,index,patient,vt,minutes,code,label,value,i
17,17,1,1991-04-24 07:52:00+00:00,4243,34,NPH insulin dose,14.0,4
18,18,1,1991-04-24 07:52:00+00:00,4243,58,Pre-breakfast blood glucose measurement,239.0,4
19,19,1,1991-04-24 07:52:00+00:00,4243,33,Regular insulin dose,10.0,4
20,20,1,1991-04-24 12:00:00+00:00,4491,33,Regular insulin dose,4.0,4


In [44]:
def hypoglicemia_from_prediction_window(dfs):
    r, ids =[], []
    for i, df in enumerate(dfs["label"]):
        ids.append(i)
        if len(df) > 0 and "Hypoglycemic symptoms" in set(df["label"]):
            r.append(True) 
        else:
            r.append(False)

    return pd.DataFrame({'Hypoglycemia':r, 'i':ids})




In [45]:
def df_to_sequences(df):
    df["sequence"] = df.apply(lambda row: (row['vt'], row['label'], row['value']), axis=1)
    r =df.groupby("i")["sequence"].agg(list).reset_index()
    for i in range(len(r)):
        r.iloc[i]["sequence"].sort(key=lambda x: x[0])
    return r

Z = df_to_sequences(final_odf)

In [46]:
CLASS_LABEL = "Hypoglycemic symptoms"

In [52]:
Z = Z[Z.apply(lambda x: CLASS_LABEL not in [y[1] for y in x.sequence], axis=1)]

In [53]:
Z = pd.merge(Z, hypoglicemia_from_prediction_window(final_pdf), on='i', how='inner')

In [54]:
Z.Hypoglycemia.mean()

0.17703349282296652

In [55]:
Z["class"] = Z["Hypoglycemia"]

In [56]:
import json
from datetime import datetime

In [57]:
ZPARAMS = {
    "main_value":main_value, 
    "main_feature":main_feature, 
    "time_feature":time_feature, 
    "timeseries_df_list":timeseries_df_list, 
    "event_df_list":event_df_list, 
    "slicer_params":slicer_params,
    "class": "Hypoglycemic symptoms"
}
TIME = datetime.now().strftime('%Y%m%d_%H%M')
ZNAME = f"{TIME}_diabetes_{main_value}_removed"


In [58]:
Z[["sequence", "class"]].to_csv(f"datasets/{ZNAME}.csv", index=False)
with open(f"datasets/{ZNAME}.json", 'w') as json_file:
    json.dump(ZPARAMS, json_file, indent=2)
